In [1]:
import pandas as pd
import sqlite3

In [2]:
# Connect to database sqlite where the information regarding the Dribbble dataset are stored.
conn = sqlite3.connect("dribbble_temporary.db", detect_types = sqlite3.PARSE_DECLTYPES) # https://docs.python.org/3/library/sqlite3.html
c = conn.cursor()

# Dribbble database (data cleaning)

This notebook deals with fixing some inconsistencies in the Dribbble database. In particular, we notice how the information regarding the users (*username*) and the shots (*shot_id*) are discrepant between the various tables. 

For example, a generic user 'X' could be present in the *users* table but not in the *shots* table where the username field is the identifier (n the *shots* table are stored the information of all the users: both those who have published and those who have not published any shots). This type of anomaly involves also the *shot_id* information among the tables where the *shot_id* field is an identifier. 

Let's find and fix them.

## Intersection between the *username*

In order to solve the anomalies regarding the *username*, we decide to consider as our Dribbble population only the users in common between the tables where the *username* field is an identifier (*users*, *followers*, *skills*, *shots*). 

In [3]:
# Table 'users'.
users = pd.read_sql("SELECT username FROM users", conn)

In [4]:
users.username.isna().sum()

0

In [5]:
len(users.username.unique())

770732

In [6]:
# Table 'followers'.
followers = pd.read_sql("SELECT destination FROM followers", conn)

In [7]:
followers.destination.isna().sum()

0

In [8]:
len(followers.destination.unique())

770767

In [9]:
# Table 'skills'.
skills = pd.read_sql("SELECT username FROM skills", conn)

In [10]:
skills.username.isna().sum()

0

In [11]:
len(skills.username.unique())

770821

In [12]:
# Table 'shots'.
shots = pd.read_sql("SELECT author_shot FROM shots", conn)

In [13]:
shots.author_shot.isna().sum()

0

In [14]:
len(shots.author_shot.unique())

770544

### Common *username*

In [15]:
intersection_users = list(set.intersection(set(users.username.unique()), set(followers.destination.unique()), set(shots.author_shot.unique()), set(skills.username.unique())))

In [16]:
len(intersection_users)

770540

### Filtering *username*

We save these users into a new table that we will use to filter the usernames into the other tables. 

N.B. We cannot directly filter using a python list because there exist a limit (`SQLITE_LIMIT_VARIABLE_NUMBER`) in the number of values we can provide to execute which is by default set to 999 `?`.

In [17]:
intersection_users = pd.DataFrame(intersection_users, columns = ["username"])

In [18]:
# Save the dataframe into the sql database.
intersection_users.to_sql("intersection_users", conn, index = False, dtype = {"username": "TEXT"})

In [19]:
c.execute("DELETE FROM users WHERE username NOT IN (SELECT username FROM intersection_users)")
conn.commit()

In [20]:
c.execute("DELETE FROM followers WHERE destination NOT IN (SELECT username FROM intersection_users)")
conn.commit()

In [21]:
c.execute("DELETE FROM followers WHERE source NOT IN (SELECT username FROM intersection_users)")
conn.commit()

In [22]:
c.execute("DELETE FROM shots WHERE author_shot NOT IN (SELECT username FROM intersection_users)")
conn.commit()

In [23]:
c.execute("DELETE FROM shots WHERE team_username NOT IN (SELECT username FROM intersection_users)")
conn.commit()

In [24]:
c.execute("DELETE FROM tags WHERE author_shot NOT IN (SELECT username FROM intersection_users)")
conn.commit()

In [25]:
c.execute("DELETE FROM skills WHERE username NOT IN (SELECT username FROM intersection_users)")
conn.commit()

In [26]:
c.execute("DELETE FROM comments WHERE author_comment NOT IN (SELECT username FROM intersection_users)")
conn.commit()

In [27]:
c.execute("DELETE FROM likes WHERE author_like NOT IN (SELECT username FROM intersection_users)")
conn.commit()

These queries allow to filter the usernames keeping the rows where the usernames have `NULL` values.

## Intersection between the *shot_id*

In order to solve the anomalies regarding the *shot_id*, we decide to consider as our Dribbble posts only the shots in common between the tables where the *shot_id* field is an identifier (*shots*, *comments*, *likes*). 

In [28]:
# Table 'shots'.
shots = pd.read_sql("SELECT shot_id FROM shots", conn)

In [29]:
len(shots.shot_id.unique())

2480528

In [30]:
# Table 'comments'.
comments = pd.read_sql("SELECT shot_id FROM comments", conn)

In [31]:
comments.shot_id.isna().sum()

0

In [32]:
len(comments.shot_id.unique())

2482681

In [33]:
# Table 'likes'.
likes = pd.read_sql("SELECT shot_id FROM likes", conn)

In [34]:
likes.shot_id.isna().sum()

0

In [35]:
len(likes.shot_id.unique())

2484398

### Common *shot_id*

In [36]:
intersection_shots = list(set.intersection(set(shots.shot_id.dropna().unique()), set(comments.shot_id.unique()), set(likes.shot_id.unique())))

In [37]:
len(intersection_shots)

2475311

### Filtering *shot_id*

We save these shots into a new table that we will use to filter the shots into the other tables. 

N.B. We cannot directly filter using a python list because there exist a limit (`SQLITE_LIMIT_VARIABLE_NUMBER`) in the number of values we can provide to execute which is by default set to 999 `?`.

In [38]:
intersection_shots = pd.DataFrame(intersection_shots, columns = ["shot_id"])

In [39]:
# Save the dataframe into the sql database.
intersection_shots.to_sql("intersection_shots", conn, index = False, dtype = {"shot_id": "INT"})

In [40]:
c.execute("DELETE FROM shots WHERE shot_id NOT IN (SELECT shot_id FROM intersection_shots)")
conn.commit()

In [41]:
c.execute("DELETE FROM comments WHERE shot_id NOT IN (SELECT shot_id FROM intersection_shots)")
conn.commit()

In [42]:
c.execute("DELETE FROM likes WHERE shot_id NOT IN (SELECT shot_id FROM intersection_shots)")
conn.commit()

In [43]:
c.execute("DELETE FROM tags WHERE shot_id NOT IN (SELECT shot_id FROM intersection_shots)")
conn.commit()

These queries allow to filter the shot_id keeping the rows when the shot_id have `NULL` values.

# Adjust tables after filtering

Some tables must be adjusted in order to create a database with a correct structure: we delete the rows with `NULL` values in all the columns except one. For example, for the *followers* table:

| destination | created_at | source |
| --- | --- | --- |
| John | NULL | NULL |


The information regarding these cases will be available directly from the features *_count* in the suitable tables. For example, `John` will have 0 `followers_count` in the *users* table. We will build this information in the next notebooks.

## Table *shots*

In [44]:
# Delete NULL rows from the 'shots' table.
c.execute("DELETE FROM shots WHERE shot_id IS NULL")
conn.commit()

## Table *skills*

In [45]:
# Delete NULL rows from the 'skills' table.
c.execute("DELETE FROM skills WHERE skill IS NULL")
conn.commit()

## Table *followers*

In [46]:
# Delete NULL rows from the 'followers' table.
c.execute("DELETE FROM followers WHERE source IS NULL")
conn.commit()

## Table *tags*

In [47]:
# Delete NULL rows from the 'tags' table.
c.execute("DELETE FROM tags WHERE tag IS NULL")
conn.commit()

## Table *comments*

In [48]:
# Delete NULL rows from the 'comments' table.
c.execute("DELETE FROM comments WHERE comment_id IS NULL")
conn.commit()

## Table *likes*

In [49]:
# Delete NULL rows from the 'likes' table.
c.execute("DELETE FROM likes WHERE like_id IS NULL")
conn.commit()

In [50]:
# Remove tables used for intersections.
c.execute("""DROP TABLE intersection_users""")
c.execute("""DROP TABLE intersection_shots""")
conn.commit()

In [51]:
# Release effectively memory from the database.
c.execute("VACUUM");

In [52]:
conn.close()